## Import

In [16]:
import pandas as pd
import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
import os
print(os.getcwd())

print(os.listdir(os.getcwd()))
# print(os.listdir('/private/var/mobile/Containers'))
path = '/private'
print(os.listdir(path))
os.chdir('/private')

/private/var/mobile/Containers/Data/Application/2CF9B4AA-DD1C-43D5-A660-EF7964DF8ECF/Documents
['.config', '.ipython', '.ipynb_checkpoints', '.jupyter']
['preboot', 'etc', 'var', 'xarts', 'system_data']


## Data Load

In [3]:
apply_train_df = pd.read_csv('apply_train.csv')
apply_train_df

,resume_seq,recruitment_seq
0,U05833,R03838
1,U06456,R02144
2,U07807,R01877
3,U04842,R02463
4,U08336,R00112
...,...,...
57941,U02270,R03430
57942,U02640,R04987
57943,U08238,R01342
57944,U01296,R06363


## User-Item Matrix / Similarity / Score

In [13]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
user_item_matrix[user_item_matrix > 1] = 1

print(user_item_matrix.shape)
# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_item_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

(8482, 6695)


## Prediction

In [4]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        recommendations.append([user, job])

## Submission

In [5]:
    # sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)